# 剪枝基础理论与实现

**SOTA 教育标准** | 包含剪枝原理、重要性评估、非结构化剪枝实现

---

## 1. 剪枝核心原理

### 1.1 为什么剪枝？

**过参数化问题**: 现代神经网络存在大量冗余参数。

**剪枝目标**:
- 减少模型大小（存储）
- 加速推理（计算）
- 降低能耗（部署）

### 1.2 剪枝分类

| 类型 | 粒度 | 硬件友好 | 压缩率 |
|:-----|:-----|:--------:|:------:|
| 非结构化 | 单个权重 | 低 | 高 |
| 结构化 | 通道/层 | 高 | 中 |
| 半结构化 | N:M 稀疏 | 中 | 中 |

### 1.3 重要性评估

**基于幅度**: $\text{Importance}(w) = |w|$

**基于梯度**: $\text{Importance}(w) = |w \cdot \nabla_w L|$

**基于 Hessian**: $\text{Importance}(w) = w^2 \cdot H_{ww}$

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

---

## 2. 非结构化剪枝实现

In [ ]:
@dataclass
class PruningConfig:
    """剪枝配置。"""
    sparsity: float = 0.5  # 目标稀疏度
    method: str = "magnitude"  # magnitude, gradient, random
    scope: str = "global"  # global, local

In [ ]:
class MagnitudePruner:
    """基于幅度的剪枝器。

    Core Idea:
        移除绝对值最小的权重。

    Mathematical Theory:
        mask = |w| > threshold
        threshold = quantile(|w|, sparsity)

    Summary:
        简单有效，是最常用的剪枝方法。
    """

    def __init__(self, model: nn.Module, config: PruningConfig):
        self.model = model
        self.config = config
        self.masks: Dict[str, Tensor] = {}

    def compute_threshold(self, weights: List[Tensor]) -> float:
        """计算全局剪枝阈值。"""
        all_weights = torch.cat([w.abs().flatten() for w in weights])
        threshold = torch.quantile(all_weights, self.config.sparsity)
        return threshold.item()

    def prune(self) -> Dict[str, float]:
        """执行剪枝。"""
        weights = [p for n, p in self.model.named_parameters() if p.dim() > 1]
        threshold = self.compute_threshold(weights)

        stats = {}
        for name, param in self.model.named_parameters():
            if param.dim() > 1:
                mask = (param.abs() > threshold).float()
                self.masks[name] = mask
                param.data *= mask
                sparsity = 1 - mask.mean().item()
                stats[name] = sparsity

        return stats
    def __init__(self, model: nn.Module, config: PruningConfig):
        self.model = model
        self.config = config
        self.masks: Dict[str, Tensor] = {}

---

## 3. 可视化分析

In [ ]:
def visualize_pruning(model: nn.Module, sparsity: float = 0.5) -> None:
    """可视化剪枝效果。"""
    # 获取第一层权重
    weight = list(model.parameters())[0].data.clone()

    # 剪枝
    threshold = torch.quantile(weight.abs().flatten(), sparsity)
    mask = (weight.abs() > threshold).float()
    weight_pruned = weight * mask

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # 原始权重分布
    axes[0].hist(weight.flatten().numpy(), bins=50, alpha=0.7)
    axes[0].axvline(-threshold.item(), color='r', linestyle='--', label='Threshold')
    axes[0].axvline(threshold.item(), color='r', linestyle='--')
    axes[0].set_title('Original Weight Distribution')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # 剪枝后分布
    axes[1].hist(weight_pruned.flatten().numpy(), bins=50, alpha=0.7, color='orange')
    axes[1].set_title(f'After Pruning ({sparsity*100:.0f}% sparse)')
    axes[1].grid(True, alpha=0.3)

    # 掩码可视化
    axes[2].imshow(mask[:32, :32].numpy(), cmap='binary')
    axes[2].set_title('Pruning Mask (32x32)')
    axes[2].colorbar(ax=axes[2], label='Keep (1) / Prune (0)')

    plt.tight_layout()
    plt.show()


# 创建新模型并可视化
model = nn.Linear(128, 64)
visualize_pruning(model, sparsity=0.7)

---

## 4. 总结

| 概念 | 公式/方法 | 说明 |
|:-----|:---------|:-----|
| **幅度剪枝** | $\text{mask} = |w| > \theta$ | 移除小权重 |
| **全局剪枝** | 所有层统一阈值 | 更均衡 |
| **局部剪枝** | 每层独立阈值 | 保持层结构 |

**关键点**:
1. 非结构化剪枝压缩率高但硬件不友好
2. 幅度剪枝简单有效
3. 剪枝后需要微调恢复精度